# Practice variety of regressions on 1 df

In [1]:
import geopandas
import intake
import matplotlib.pyplot as plt
import numpy as np
import pandas
import sklearn.ensemble
import sklearn.inspection
import sklearn.preprocessing
import sklearn.linear_model
import sklearn.pipeline
import sklearn.utils

import laplan
import utils

cat = intake.open_catalog("../catalogs/*.yml")

In [ ]:
tracts = cat.census_tracts.read()

joined = geopandas.read_parquet("../data/toc_df_for_regression.parquet")

## Linear regression
Do a linear regression with just 1 variable, plot to see how line is fitted.

Make map of actual vs predicted.

In [ ]:
linear_model = sklearn.linear_model.LinearRegression(fit_intercept=True, normalize=False)

In [ ]:
linear_variables = [
    "pct_pop_renter",
]
target = "TOC"

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9,9))
for i, var in enumerate(linear_variables + ["TOC"]):
    ax = axes.ravel()[i]
    joined.reset_index().plot(ax=ax, column=var)
    ax.axis("off")
    ax.set_title(var)

In [ ]:
fit_me = (joined.dropna()
          .assign(
            TOC = joined.TOC.astype(float)
        )[joined.pct_eligible_zoning > 0]
    [linear_variables + [target, "geometry"]]
)

In [ ]:
linear_model.fit(fit_me[linear_variables], fit_me["TOC"])

In [ ]:
print(linear_model.coef_)
print(linear_model.intercept_)

In [ ]:
linear_model_yhat = linear_model.predict(fit_me[linear_variables])

In [ ]:
yhat = pandas.DataFrame(linear_model_yhat).rename(columns = {0: "yhat"})
with_yhat = pandas.merge(
                fit_me.reset_index(), 
                yhat, 
                left_index=True, 
                right_index=True)

with_yhat.head()

In [ ]:
plt.scatter(with_yhat.pct_pop_renter, with_yhat.TOC, color = "orange")
plt.plot(with_yhat.pct_pop_renter, with_yhat.yhat)

In [ ]:
map_me = pandas.merge(tracts, with_yhat.drop(columns = "geometry"), 
                       on = "GEOID", how = "left", validate = "1:1")

map_me = map_me.assign(
    yhat = map_me.yhat.fillna(0)
)

In [ ]:
# Plot maps of actual TOC vs predicted TOC
def make_map(predictions):
    cmap="plasma"
    fig, axes = plt.subplots(1,2, figsize=(9, 9))
    axes[0].axis("off")
    axes[0].set_title("Actual TOC")
    
    map_me.plot(ax=axes[0], column = "TOC", cmap = cmap)
    
    axes[1].axis("off")
    axes[1].set_title("Predicted TOC")
     
    map_me.plot(ax=axes[1], column="yhat", cmap=cmap)
    
    plt.close(fig)
    return fig

In [ ]:
make_map(with_yhat)

## Logistic Regression

Do logistic regression trying to predict `toc_AIN`, which is 0/1.

In [ ]:
logistic_model = sklearn.linear_model.LogisticRegression(fit_intercept=True)

In [ ]:
logistic_variables = [
    "pct_eligible_zoning"
]
target = "toc_AIN"

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9,4))
for i, var in enumerate(logistic_variables + ["toc_AIN"]):
    # Can also designate which specific grid if we wanted to do a 2x3 grid
    #ax1 = plt.subplot2grid((3,2),(0, 0))
    ax = axes.ravel()[i]
    joined.reset_index().plot(ax=ax, column=var)
    ax.axis("off")
    ax.set_title(var)

In [ ]:
logistic_model.fit(joined[logistic_variables], joined["toc_AIN"])

In [ ]:
logistic_model_yhat = logistic_model.predict(joined[logistic_variables])

In [ ]:
yhat = pandas.DataFrame(logistic_model_yhat).rename(columns = {0: "yhat"})
with_yhat = pandas.merge(
                joined.reset_index(), 
                yhat, 
                left_index=True, 
                right_index=True)

with_yhat.head()

In [ ]:
map_me = pandas.merge(tracts, with_yhat.drop(columns = "geometry"), 
                       on = "GEOID", how = "left", validate = "1:1")

map_me = map_me.assign(
    yhat = map_me.yhat.fillna(0)
)

In [ ]:
# Plot maps of actual TOC vs predicted TOC
def make_map(predictions):
    cmap="plasma"
    fig, axes = plt.subplots(1,2, figsize=(9, 9))
    axes[0].axis("off")
    axes[0].set_title("Actual toc_AIN")
    
    map_me.plot(ax=axes[0], column = "toc_AIN", cmap = cmap)
    
    axes[1].axis("off")
    axes[1].set_title("Predicted toc_AIN")
     
    map_me.plot(ax=axes[1], column="yhat", cmap=cmap)
    
    plt.close(fig)
    return fig

In [ ]:
make_map(with_yhat)

## Random Forest

In [ ]:
variables = [
    "medhhincome",
    "pct_pop_renter",
    "pct_zero_veh_workers",
    "density",
    "pct_whitenonhisp",
    "pct_eligible_zoning", 
    "Tier_1",
    "Tier_2",
    "Tier_3",
    "Tier_4",
]
target = "TOC"

In [ ]:
to_fit = joined[
    (joined.Tier_1 > 0) |
    (joined.Tier_2 > 0) |
    (joined.Tier_3 > 0) |
    (joined.Tier_4 > 0)
]

In [ ]:
# Set a max depth to avoid over-fitting
random_forest_model = sklearn.pipeline.Pipeline([
    ("scaler", sklearn.preprocessing.StandardScaler(with_mean=True)),
    ("regressor", sklearn.ensemble.RandomForestRegressor(max_depth=10)),
])

In [ ]:
# VERY SMALL CHANGES (original tracts already has index set to GEOID)
def plot_model(predictions):
    vmin=0
    vmax=0.15
    cmap="plasma"
    fig, axes = plt.subplots(1,2, figsize=(16, 16))
    axes[0].axis("off")
    axes[0].set_title("Actual TOC/parcel")
    (tracts.set_index("GEOID")
     .assign(
         TOC=to_fit.TOC,
         # MINOR CHANGE HERE
         norm_TOC=to_fit.TOC.divide(to_fit.total_AIN),
     ).fillna({"TOC": 0, "norm_TOC": 0})
     .plot(ax=axes[0],column="norm_TOC", vmax=vmax, vmin=vmin, cmap=cmap)
    )
    axes[1].axis("off")
    axes[1].set_title("Predicted TOC/parcel")
    (tracts.set_index("GEOID")
     .assign(
         predictions=pandas.Series(predictions, index=to_fit.index),
         # MINOR CHANGE HERE
         norm_pred=pandas.Series(predictions, index=to_fit.index).divide(to_fit.total_AIN),
     )
     .fillna({"predictions": 0, "norm_pred": 0})
     .plot(ax=axes[1],column="norm_pred", cmap=cmap, vmin=vmin, vmax=vmax)
    )
    plt.close(fig)
    return fig

In [ ]:
np.random.seed(1)

random_forest_model.fit(to_fit[variables], to_fit[target])
plot_model(random_forest_model.predict(to_fit[variables]))

In [ ]:
print(f"Score: {random_forest_model.score(to_fit[variables], to_fit[target])}")
pandas.Series(
    random_forest_model["regressor"].feature_importances_,
    index=variables,
).sort_values(ascending=False)

In [ ]:
imp = sklearn.inspection.permutation_importance(
    random_forest_model,
    to_fit[variables],
    to_fit[target]
)
pandas.Series(imp["importances_mean"], index=variables)

In [ ]:
feature_importance = pandas.Series(
    random_forest_model["regressor"].feature_importances_,
    index=variables,
).sort_values(ascending=False).to_frame("feature_importance")

importances_mean = pandas.Series(imp["importances_mean"], index=variables).to_frame("importances_mean")

In [ ]:
feature_importance.merge(
    importances_mean, 
    left_index=True, 
    right_index=True).sort_values("feature_importance")

In [ ]:
# Try without Tier_* variables
np.random.seed(1)
variables2 = [
    "medhhincome",
    "pct_pop_renter",
    "pct_zero_veh_workers",
    "density",
    "pct_whitenonhisp",
    "pct_eligible_zoning", 
]

target = "TOC"

random_forest_model.fit(to_fit[variables2], to_fit[target])
plot_model(random_forest_model.predict(to_fit[variables2]))

In [ ]:
print(f"Score: {random_forest_model.score(to_fit[variables2], to_fit[target])}")
feature_importance = pandas.Series(
    random_forest_model["regressor"].feature_importances_,
    index=variables2,
).sort_values(ascending=False).to_frame("feature_importance")

imp = sklearn.inspection.permutation_importance(
    random_forest_model,
    to_fit[variables2],
    to_fit[target]
)

importances_mean = pandas.Series(imp["importances_mean"], index=variables2).to_frame("importances_mean")

In [ ]:
feature_importance.merge(
    importances_mean, 
    left_index=True, 
    right_index=True).sort_values("feature_importance")

In [ ]:
np.random.seed(1)
fewer_variables = [
    "Tier_3",
    "Tier_2",
    "pct_pop_renter",
]

random_forest_model.fit(to_fit[fewer_variables], to_fit[target])
plot_model(random_forest_model.predict(to_fit[fewer_variables]))

In [ ]:
print(f"Score: {random_forest_model.score(to_fit[fewer_variables], to_fit[target])}")
feature_importance = pandas.Series(
    random_forest_model["regressor"].feature_importances_,
    index=fewer_variables
).sort_values(ascending=False).to_frame("feature_importance")

imp = sklearn.inspection.permutation_importance(
    random_forest_model,
    to_fit[fewer_variables],
    to_fit[target]
)
importance_means = pandas.Series(imp["importances_mean"], index=fewer_variables).to_frame("importance_means")

In [ ]:
feature_importance.merge(
    importances_mean, 
    left_index=True, 
    right_index=True).sort_values("feature_importance")

## Poisson Regression

In [ ]:
poisson_model = sklearn.pipeline.Pipeline([
    ("scaler", sklearn.preprocessing.StandardScaler(with_mean=True)),
    ("regressor", sklearn.linear_model.PoissonRegressor(fit_intercept=True, alpha=0, tol=1.e-6))
])

In [ ]:
poisson_model.fit(to_fit[fewer_variables], to_fit[target])
plot_model(poisson_model.predict(to_fit[fewer_variables]))

In [ ]:
print(f"Poisson Score: {poisson_model.score(to_fit[fewer_variables], to_fit[target].astype('int64'))}")
imp = sklearn.inspection.permutation_importance(
    poisson_model,
    to_fit[fewer_variables],
    to_fit[target].astype("int64")
)
pandas.Series(imp["importances_mean"], index=fewer_variables)

In [ ]:
poisson_model.fit(to_fit[fewer_variables], to_fit[target].astype('int64'))

In [ ]:
poisson_model_yhat = poisson_model.predict(to_fit[fewer_variables]).astype("int64")

In [ ]:
yhat = pandas.DataFrame(poisson_model_yhat).rename(columns = {0: "yhat"})
with_yhat = pandas.merge(
                joined.reset_index(), 
                yhat, 
                left_index=True, 
                right_index=True)

with_yhat.head()

In [ ]:
map_me = pandas.merge(tracts, with_yhat.drop(columns = "geometry"), 
                       on = "GEOID", how = "left", validate = "1:1")

map_me = map_me.assign(
    TOC = map_me.TOC.fillna(0),
    yhat = map_me.yhat.fillna(0)
)

In [ ]:
# Plot maps of actual TOC vs predicted TOC
def make_map(predictions):
    cmap="plasma"
    fig, axes = plt.subplots(1,2, figsize=(9, 9))
    axes[0].axis("off")
    axes[0].set_title("Actual TOC")
    
    map_me.plot(ax=axes[0], column = "TOC", cmap = cmap)
    
    axes[1].axis("off")
    axes[1].set_title("Predicted TOC")
     
    map_me.plot(ax=axes[1], column="yhat", cmap=cmap)
    
    plt.close(fig)
    return fig

In [ ]:
make_map(map_me)

In [ ]:
def sample_poisson_model(data, target, norm=1, seed=1): 
    def get_coefs(model):
        return model["regressor"].coef_
    rs = np.random.RandomState(seed)
    samples = np.array([
        get_coefs(
            poisson_model.fit(
                *sklearn.utils.resample(
                    data,
                    (target/norm).replace([np.nan, np.inf], 0.0),
                    random_state=rs,
                )
            )
        )
        for i in range(1000)
    ])
    return samples

In [ ]:
subset = fewer_variables
samples = sample_poisson_model(to_fit[subset], to_fit[target])

fig, axes = plt.subplots(
    len(subset),
    1,
    sharex=True,
    sharey=True,
    figsize=(8,12),

)

for i, var in enumerate(subset):
    ax = axes[i]
    ax.violinplot(samples[:,i], vert=False)
    ax.set_ylabel(var)
    ax.axvline(0, color="maroon", ls="--")

In [ ]:
scaler = poisson_model["scaler"]
regressor = poisson_model["regressor"]

print(scaler)
print(regressor)

In [ ]:
beta = pandas.Series(regressor.coef_/scaler.scale_, index=fewer_variables)
alpha = regressor.intercept_ - np.dot(scaler.mean_, beta)
print(beta)
print(alpha)

In [ ]:
# Back out the coefficients to be interpretable as incidence rate ratios
# Apply e^(coeff) to get it
# https://stats.idre.ucla.edu/stata/output/poisson-regression/
import IPython.display

scale = 100
val = (np.exp(beta[0]*scale)-1.0)*100.
display(IPython.display.Markdown(
    f"For every {scale:,} Tier 3 parcels, "
    f"there is a {val:.0f}% increase in TOC entitlements"
))
scale = 100
val = (np.exp(beta[1]*scale)-1.0)*100.
display(IPython.display.Markdown(
    f"For every {scale:,} Tier 2 parcels, "
    f"there is a {val:.0f}% increase in TOC entitlements"
))
scale = 0.1
val = (np.exp(beta[2]*scale)-1)*100.
display(IPython.display.Markdown(
    f"For every {100*scale:g}% increase in population of renters, "
    f"there is a {val:.0f}% increase in TOC entitlements"
))

In [ ]:
coeff1 = -.0035232 
np.exp(coeff1)